# HLLSet Manifold - Refactored System Demo

This notebook demonstrates the refactored three-layer architecture:

1. **Kernel**: Stateless pure morphisms (absorb, union, intersection, difference)
2. **HRT**: Immutable PyTorch-based data structure
3. **Evolution**: Three-state model (In-Process → Current → History)

## Key Concepts

- **Immutability**: All modifications create new objects with new hashes
- **Content-addressing**: All objects named by SHA1 of content (no UUIDs, no randomness)
- **Evolution**: System progresses through discrete steps (each step appends to history)
- **Time Travel**: Can checkout historical states from Git and create new branches from them
- **Git integration**: History pushed to Git, state is reconstructible from commit hash

In [1]:
# Setup
from core import (
    Kernel, HLLSet,
    ImmutableTensor, TensorEvolution, TensorEvolutionTriple,
    HRT, HRTConfig, HRTEvolution, HRTEvolutionTriple,
    compute_element_hash, compute_aggregate_hash, compute_structural_hash,
    record_operation
)

import torch
import numpy as np
print("✓ Imports successful")

✓ Imports successful


---

## Layer 1: Kernel (Pure Morphisms)

The kernel is a **stateless transformation engine**. It has:
- No storage (no CAS, no history)
- No side effects
- Pure functions: same input → same output

Morphism types (all return new HLLSets):
- `absorb`: Set[str] → HLLSet
- `union`: HLLSet × HLLSet → HLLSet
- `intersection`: HLLSet × HLLSet → HLLSet
- `difference`: HLLSet × HLLSet → HLLSet
- `add`: HLLSet × tokens → HLLSet

In [2]:
# Create kernel
kernel = Kernel(p_bits=10)

print("="*70)
print("KERNEL: Stateless Pure Morphisms")
print("="*70)

# Morphism 1: absorb
print("\n1. absorb: Set[str] → HLLSet")
h1 = kernel.absorb({'cat', 'dog', 'bird', 'fish'})
h2 = kernel.absorb({'dog', 'fish', 'whale', 'shark'})
print(f"   A = absorb({{'cat', 'dog', 'bird', 'fish'}})")
print(f"       → {h1}")
print(f"   B = absorb({{'dog', 'fish', 'whale', 'shark'}})")
print(f"       → {h2}")

# Morphism 2: union
print("\n2. union: HLLSet × HLLSet → HLLSet")
h_union = kernel.union(h1, h2)
print(f"   A ∪ B = {h_union}")

# Morphism 3: intersection
print("\n3. intersection: HLLSet × HLLSet → HLLSet")
h_inter = kernel.intersection(h1, h2)
print(f"   A ∩ B = {h_inter}")

# Morphism 4: difference
print("\n4. difference: HLLSet × HLLSet → HLLSet")
h_diff = kernel.difference(h1, h2)
print(f"   A \ B = {h_diff}")

# Key property: kernel is stateless
print("\n5. Stateless Property")
print(f"   - No storage of previous results")
print(f"   - Same input always produces same output")
print(f"   - A == absorb({{'cat', 'dog', 'bird', 'fish'}}): {h1 == kernel.absorb({'cat', 'dog', 'bird', 'fish'})}")

KERNEL: Stateless Pure Morphisms

1. absorb: Set[str] → HLLSet
   A = absorb({'cat', 'dog', 'bird', 'fish'})
       → HLLSet(ae6849ae..., |A|≈4.0)
   B = absorb({'dog', 'fish', 'whale', 'shark'})
       → HLLSet(95315465..., |A|≈4.0)

2. union: HLLSet × HLLSet → HLLSet
   A ∪ B = HLLSet(204038e2..., |A|≈8.0)

3. intersection: HLLSet × HLLSet → HLLSet
   A ∩ B = HLLSet(63638e5d..., |A|≈0.0)

4. difference: HLLSet × HLLSet → HLLSet
   A \ B = HLLSet(d6f1bd9c..., |A|≈0.0)

5. Stateless Property
   - No storage of previous results
   - Same input always produces same output
   - A == absorb({'cat', 'dog', 'bird', 'fish'}): True


<>:30: SyntaxWarning: invalid escape sequence '\ '
<>:30: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipykernel_19524/1159923313.py:30: SyntaxWarning: invalid escape sequence '\ '
  print(f"   A \ B = {h_diff}")


### Kernel Immutability

The kernel never modifies HLLSets - it always creates new ones.

In [3]:
# Demonstrate immutability
original = kernel.absorb({'a', 'b', 'c'})
print(original)
modified = kernel.add(original, 'x')

print(f"Original:  {original.name[:24]}... (|A|≈{original.cardinality():.0f})")
print(f"Modified:  {modified.name[:24]}... (|A|≈{modified.cardinality():.0f})")
print(f"Original unchanged: {original.name == kernel.absorb({'a', 'b', 'c'}).name}")
print(f"\nOriginal and modified are different objects: {original.name != modified.name}")

HLLSet(f8f7617a..., |A|≈3.0)
Original:  f8f7617a4427d20222f49380... (|A|≈3)
Modified:  8dea25ade3b0723a4c3a0b18... (|A|≈4)
Original unchanged: True

Original and modified are different objects: True


---

## Layer 2: Immutable Tensor (PyTorch Backend)

The `ImmutableTensor` provides the foundation for all data structures:

- PyTorch tensors with **clone-on-modify** semantics
- Content-addressed naming (SHA1)
- Parent pointers for lineage tracking
- GPU-ready (tensors can be moved to CUDA)

In [4]:
print("="*70)
print("IMMUTABLE TENSOR: PyTorch Backend")
print("="*70)

# Genesis tensor
print("\n1. Genesis Tensor")
t_genesis = ImmutableTensor.zeros(100, 100)
print(f"   T₀ = zeros(100, 100)")
print(f"      name = {t_genesis.name}")
print(f"      shape = {t_genesis.shape}")

# All operations create new tensors
print("\n2. Clone-on-Modify Operations")
t1 = t_genesis.with_value((50, 50), 1.0)
t2 = t_genesis.with_value((25, 25), 2.0)
print(f"   T₁ = T₀.with_value((50, 50), 1.0)")
print(f"      name = {t1.name}")
print(f"   T₂ = T₀.with_value((25, 25), 2.0)")
print(f"      name = {t2.name}")

# Merge operation
print("\n3. Merge Operation (element-wise max)")
t_merged = t1.maximum(t2)
print(f"   T₃ = maximum(T₁, T₂)")
print(f"      name = {t_merged.name}")
print(f"   Value at (50, 50): {t_merged.data[50, 50].item()}")
print(f"   Value at (25, 25): {t_merged.data[25, 25].item()}")

# Immutability guarantee
print("\n4. Immutability Guarantee")
print(f"   T₀ unchanged: {t_genesis.name}")
print(f"   T₀ == original genesis: {t_genesis.name == t_genesis.name}")

IMMUTABLE TENSOR: PyTorch Backend

1. Genesis Tensor
   T₀ = zeros(100, 100)
      name = 8287b034977e0ba3958ec54705e16ba3ee3d30b0
      shape = (100, 100)

2. Clone-on-Modify Operations
   T₁ = T₀.with_value((50, 50), 1.0)
      name = e414917af9612a921db8384b78798a36f812e4b6
   T₂ = T₀.with_value((25, 25), 2.0)
      name = c377255c3a355d3681b29aa9352f0c8d45eb4131

3. Merge Operation (element-wise max)
   T₃ = maximum(T₁, T₂)
      name = dc2c95dfc1b9a7dee2c4b37df57089391446e122
   Value at (50, 50): 1.0
   Value at (25, 25): 2.0

4. Immutability Guarantee
   T₀ unchanged: 8287b034977e0ba3958ec54705e16ba3ee3d30b0
   T₀ == original genesis: True


### Tensor Evolution Triple

The generic `TensorEvolution` class manages three-state evolution.

In [5]:
# Create evolution manager
print("\n5. Tensor Evolution Manager")
evolution = TensorEvolution(t_genesis)
print(f"   Initial triple: {evolution.triple}")

# Ingest new data
new_data = ImmutableTensor.zeros(100, 100).with_value((10, 10), 5.0)
evolution.ingest(new_data)
print(f"   After ingest: {evolution.triple}")

# Evolve
def merge_fn(in_proc, curr):
    return in_proc.maximum(curr)

def commit_fn(tensor):
    print(f"     Committing tensor {tensor.name[:20]}...")
    return tensor.name

evolution.evolve(merge_fn, commit_fn)
print(f"   After evolve: {evolution.triple}")
print(f"   Lineage: {[h[:16] + '...' for h in evolution.get_lineage()]}")


5. Tensor Evolution Manager
   Initial triple: TensorEvolutionTriple(step=0, in_process=None, current=8287b034..., history=8287b034...)
   After ingest: TensorEvolutionTriple(step=0, in_process=exists, current=8287b034..., history=8287b034...)
     Committing tensor 8287b034977e0ba3958e...
   After evolve: TensorEvolutionTriple(step=1, in_process=None, current=ba2f4654..., history=8287b034...)
   Lineage: ['8287b034977e0ba3...', '8287b034977e0ba3...']


---

## Layer 3: HRT Evolution (Three-State Model)

The **Hash Relational Tensor (HRT)** combines:
- **Adjacency Matrix (AM)**: Relationship structure using `ImmutableTensor`
- **HLLSet Lattice W**: Basic HLLSets for composition
- **Covers**: Optimal compositions (optional cache)

### Evolution Cycle

```
Step 1: Ingest → Set in_process
Step 2: Evolve → Merge in_process+current, commit current
Step 3: Cycle repeats
```

The shift to future: In-Process becomes Current, Current goes to History.

In [6]:
print("="*70)
print("HRT EVOLUTION: Three-State Model")
print("="*70)

# Setup
kernel = Kernel(p_bits=8)
config = HRTConfig(p_bits=8, h_bits=16)

print(f"\nConfiguration:")
print(f"   Dimension: {config.dimension}")
print(f"   Basic HLLSets: {config.num_basic_hllsets}")

# Create evolution manager
print("\n1. Genesis State")
evolution = HRTEvolution(config)
genesis = evolution.get_current()
print(f"   HRT₀ (genesis)")
print(f"      name = {genesis.name}")
print(f"      step = {genesis.step_number}")
print(f"      am_entries = {len(genesis.am.nonzero_entries())}")

HRT EVOLUTION: Three-State Model

Configuration:
   Dimension: 4098
   Basic HLLSets: 8196

1. Genesis State
   HRT₀ (genesis)
      name = c5b23190b25101327d01b0fa744174429d53394c
      step = 0
      am_entries = 0


In [7]:
# Evolution Cycle 1
print("\n2. Evolution Cycle 1: Ingest → Merge → Commit")

# Step 1: Ingest
data1 = {
    'camera': {'red', 'green', 'blue', 'yellow'},
    'microphone': {'low', 'mid', 'high'}
}
print(f"   Ingest: {list(data1.keys())}")
evolution.ingest(data1, kernel)
print(f"   In-process HRT created")
print(f"      AM entries: {len(evolution.triple.in_process.am.nonzero_entries())}")

# Step 2: Evolve (merge + commit)
def mock_commit(hrt):
    """Mock Git commit - would save to persistent store."""
    return hrt.name

evolution.evolve(kernel, mock_commit)

hrt1 = evolution.get_current()
print(f"   After evolve:")
print(f"      HRT₁ name = {hrt1.name}")
print(f"      step = {hrt1.step_number}")
print(f"      am_entries = {len(hrt1.am.nonzero_entries())}")
print(f"      parent = {hrt1.parent_hrt[:32] if hrt1.parent_hrt else 'genesis'}...")


2. Evolution Cycle 1: Ingest → Merge → Commit
   Ingest: ['camera', 'microphone']
   In-process HRT created
      AM entries: 7
   After evolve:
      HRT₁ name = b35a69fe70eaeabdd032395b2da5df1cd1038a5f
      step = 2
      am_entries = 7
      parent = c0aff2fb6d16dc0abbde3256a7ae6e69...


In [8]:
# Evolution Cycle 2
print("\n3. Evolution Cycle 2")
data2 = {
    'thermometer': {'hot', 'warm', 'cold'},
    'pressure': {'high_p', 'low_p'}
}
print(f"   Ingest: {list(data2.keys())}")
evolution.ingest(data2, kernel)
evolution.evolve(kernel, mock_commit)

hrt2 = evolution.get_current()
print(f"   After evolve:")
print(f"      HRT₂ name = {hrt2.name}")
print(f"      step = {hrt2.step_number}")
print(f"      am_entries = {len(hrt2.am.nonzero_entries())}")


3. Evolution Cycle 2
   Ingest: ['thermometer', 'pressure']
   After evolve:
      HRT₂ name = 2f6f90e84ae6312e27d4e28ecd3b4a59a9c9ded7
      step = 3
      am_entries = 12


In [9]:
# Lineage
print("\n4. Lineage (Commit History)")
lineage = evolution.get_lineage()
print(f"   History chain:")
for i, h in enumerate(lineage):
    marker = " ← HEAD" if i == len(lineage) - 1 else ""
    print(f"      [{i}] {h}{marker}")


4. Lineage (Commit History)
   History chain:
      [0] genesis
      [1] c5b23190b25101327d01b0fa744174429d53394c
      [2] b35a69fe70eaeabdd032395b2da5df1cd1038a5f ← HEAD


In [10]:
# Projections
print("\n5. Future/Past Projections")
current_hrt = evolution.get_current()

# Future projection: columns → rows
future = current_hrt.project_future([2, 3, 4, 5])
print(f"   Future projection (cols → rows): shape {future.shape}")
print(f"      Top activations: {torch.topk(future.data, 3).values.tolist()}")

# Past projection: rows → columns
past = current_hrt.project_past([0, 1, 2])
print(f"   Past projection (rows → cols): shape {past.shape}")
print(f"      Top activations: {torch.topk(past.data, 3).values.tolist()}")


5. Future/Past Projections
   Future projection (cols → rows): shape (4098,)
      Top activations: [0.0, 0.0, 0.0]
   Past projection (rows → cols): shape (4098,)
      Top activations: [0.0, 0.0, 0.0]


### HRT Immutability

All HRT operations create new HRTs. The original is never modified.

In [ ]:
print("\n6. HRT Immutability Verification")
current = evolution.get_current()
original_name = current.name

# Try to "modify" the AM
modified_am = current.am.with_entry(0, 0, 5.0)

print(f"   Original HRT name: {current.name[:40]}...")
print(f"   Modified AM name:  {modified_am.name[:40]}...")
print(f"   Names different:   {current.name != modified_am.name}")
print(f"   Original unchanged: {current.name == original_name}")

# The HRT with modified AM would be a new HRT
new_hrt = HRT(
    config=current.config,
    am=modified_am,
    lattice=current.lattice,
    parent_hrt=current.name
)
print(f"\n   New HRT from modified AM:")
print(f"      name = {new_hrt.name[:40]}...")
print(f"      parent = {new_hrt.parent_hrt[:40]}...")

---

## Content Addressing

All elements in the system are named by their content. No randomness, no UUIDs.

In [11]:
print("="*70)
print("CONTENT ADDRESSING")
print("="*70)

print("\n1. Two Types of Hashes")
print("   - Element hash: 32/64-bit (for tokens)")
print("   - Aggregate hash: SHA1 (for HLLSets, Tensors, HRTs)")

# Element hash
print("\n2. Element Hash Example")
h1 = compute_element_hash("hello", bits=64)
h2 = compute_element_hash("hello", bits=64)
h3 = compute_element_hash("world", bits=64)
print(f"   hash('hello') = {h1}")
print(f"   hash('hello') = {h2}")
print(f"   hash('world') = {h3}")
print(f"   Deterministic: {h1 == h2}")

# Aggregate hash
print("\n3. Aggregate Hash Example")
kernel = Kernel()
hll1 = kernel.absorb({'a', 'b', 'c'})
hll2 = kernel.absorb({'a', 'b', 'c'})
hll3 = kernel.absorb({'x', 'y', 'z'})
print(f"   HLLSet({{'a', 'b', 'c'}}) = {hll1.name}")
print(f"   HLLSet({{'a', 'b', 'c'}}) = {hll2.name}")
print(f"   HLLSet({{'x', 'y', 'z'}}) = {hll3.name}")
print(f"   Same content → same hash: {hll1.name == hll2.name}")
print(f"   Different content → different hash: {hll1.name != hll3.name}")

# Structural hash
print("\n4. Structural Hash (for composed objects)")
name1 = compute_structural_hash("am_hash_abc", "lattice_hash_xyz")
name2 = compute_structural_hash("am_hash_abc", "lattice_hash_xyz")
name3 = compute_structural_hash("am_hash_def", "lattice_hash_xyz")
print(f"   struct('abc', 'xyz') = {name1}")
print(f"   struct('abc', 'xyz') = {name2}")
print(f"   struct('def', 'xyz') = {name3}")
print(f"   Deterministic: {name1 == name2}")

CONTENT ADDRESSING

1. Two Types of Hashes
   - Element hash: 32/64-bit (for tokens)
   - Aggregate hash: SHA1 (for HLLSets, Tensors, HRTs)

2. Element Hash Example
   hash('hello') = 6489091345076386088
   hash('hello') = 6489091345076386088
   hash('world') = 978537718455108403
   Deterministic: True

3. Aggregate Hash Example
   HLLSet({'a', 'b', 'c'}) = f8f7617a4427d20222f49380ad4a8413c4148773
   HLLSet({'a', 'b', 'c'}) = f8f7617a4427d20222f49380ad4a8413c4148773
   HLLSet({'x', 'y', 'z'}) = 3e5d420e44b7f90e24c63c2369481f669805d7cd
   Same content → same hash: True
   Different content → different hash: True

4. Structural Hash (for composed objects)
   struct('abc', 'xyz') = eea44ee069641e541fe7af85a1916ee6d80ff7b3
   struct('abc', 'xyz') = eea44ee069641e541fe7af85a1916ee6d80ff7b3
   struct('def', 'xyz') = 8dffdbb48eda71625afa4680a1854c8520348c79
   Deterministic: True


---

## Cover Computation

A **Cover** represents an HLLSet as a composition of basic HLLSets from the lattice.

In [12]:
print("="*70)
print("COVER COMPUTATION")
print("="*70)

# Get current HRT and create a target HLLSet
hrt = evolution.get_current()
target = kernel.absorb({'red', 'green', 'hot', 'high_p'})

print(f"\nTarget HLLSet: {target}")

# Compute cover
cover = hrt.compute_cover(target, kernel)

print(f"\nComputed Cover:")
print(f"   target_hash: {cover.target_hash}")
print(f"   size: {cover.size}")
print(f"   row_indices: {sorted(cover.row_indices)[:10]}{'...' if len(cover.row_indices) > 10 else ''}")
print(f"   col_indices: {sorted(cover.col_indices)[:10]}{'...' if len(cover.col_indices) > 10 else ''}")
print(f"   cover_hash: {cover.name}")

# Add cover to HRT
hrt_with_cover = hrt.with_cover(cover)
print(f"\nHRT with cached cover:")
print(f"   covers: {len(hrt_with_cover.covers)}")
print(f"   new HRT name: {hrt_with_cover.name[:40]}...")

COVER COMPUTATION

Target HLLSet: HLLSet(e4fe5d58..., |A|≈4.0)

Computed Cover:
   target_hash: e4fe5d58986e895e4e01486686531424d1205dbc
   size: 0
   row_indices: []
   col_indices: []
   cover_hash: b41876d24755d14bd17bd803ce6c724cc84dc6f2

HRT with cached cover:
   covers: 1
   new HRT name: 2f6f90e84ae6312e27d4e28ecd3b4a59a9c9ded7...


---

## Time Travel (History Navigation)

**Important distinction:**
- **Evolution steps are irreversible** - you cannot "undo" a merge (like you cannot undo a Git commit)
- **But you can time travel** - checkout any historical state and start a new branch from there

This is exactly like Git:


The original timeline continues unchanged. Time travel creates a **new parallel timeline**.

### Implementation
- Checkout: Load HRT from Git by hash → create new Evolution manager from it
- Branch: Continue evolution from that historical point
- Merge: Later, branches can be merged (if needed)

In [13]:
print("="*70)
print("TIME TRAVEL: Checkout Historical State")
print("="*70)

# Current timeline
print("1. Current Timeline (Linear Evolution)")
lineage = evolution.get_lineage()
print(f"   Number of states: {len(lineage)}")
print(f"   Current HEAD: {lineage[-1][:24]}...")
print(f"   Current step: {evolution.get_current().step_number}")

# Simulate time travel: checkout historical state
print("2. Time Travel: Checkout Historical State")
historical_hash = lineage[1]  # Go back to state 1
print(f"   Target state: {historical_hash[:24]}...")

# In real system, this would load from Git:
# historical_hrt = git_checkout(historical_hash)
# For demo, we use the evolution manager's reference
print("   (In real system: git checkout <hash>)")

# Create new evolution branch from historical state
print("3. Create New Branch from Historical State")

# Get the historical HRT (in real system, load from Git)
historical_hrt = evolution.get_current()  # Simulating loaded state

# Create new evolution manager starting from this state
branch_evolution = HRTEvolution(config, genesis_hrt=historical_hrt)
print(f"   New branch created from step {branch_evolution.get_current().step_number}")

# Continue evolution on new branch
print("4. Continue Evolution on New Branch")
branch_data = {"alt_sensor": {"alpha", "beta", "gamma"}}
branch_evolution.ingest(branch_data, kernel)
branch_evolution.evolve(kernel, mock_commit)

print(f"   Branch current step: {branch_evolution.get_current().step_number}")
print(f"   Branch HRT: {branch_evolution.get_current().name[:24]}...")

# Original timeline unchanged
print("5. Original Timeline Unchanged")
print(f"   Original HEAD still at: {evolution.get_current().name[:24]}...")
print(f"   Original step: {evolution.get_current().step_number}")

print("✓ Time travel creates parallel timeline, doesn't alter history")

TIME TRAVEL: Checkout Historical State
1. Current Timeline (Linear Evolution)
   Number of states: 3
   Current HEAD: b35a69fe70eaeabdd032395b...
   Current step: 3
2. Time Travel: Checkout Historical State
   Target state: c5b23190b25101327d01b0fa...
   (In real system: git checkout <hash>)
3. Create New Branch from Historical State
   New branch created from step 3
4. Continue Evolution on New Branch
   Branch current step: 4
   Branch HRT: 315cd16a737444a5eca5eb3e...
5. Original Timeline Unchanged
   Original HEAD still at: 2f6f90e84ae6312e27d4e28e...
   Original step: 3
✓ Time travel creates parallel timeline, doesn't alter history


### Visual Representation of Time Travel



---

## System Invariants

Summary of the key invariants maintained by the system.

In [14]:
print("="*70)
print("SYSTEM INVARIANTS")
print("="*70)

print("\n✓ 1. Immutability")
print("   - Objects never change after creation")
print("   - All 'modifications' create new objects")
print("   - Parent pointers link old to new")

print("\n✓ 2. Idempotency")
print("   - Same inputs → same outputs")
print("   - No side effects in kernel operations")
print("   - Replaying operations yields same result")

print("\n✓ 3. Content Addressing")
print("   - Names are hashes of content")
print("   - No randomness, no UUIDs")
print("   - Deterministic across time and space")

print("\n✓ 4. Evolution Model")
print("   - Three states: In-Process → Current → History")
print("   - Shift to future: commit current, merge in-process")
print("   - History is in Git, not in memory")

print("\n✓ 5. Kernel Statelessness")
print("   - Kernel has no storage")
print("   - Pure functions only")
print("   - All state in HRT (managed by OS)")

print("\n✓ 6. PyTorch Backend")
print("   - Tensors support GPU acceleration")
print("   - Clone-on-modify ensures immutability")
print("   - Efficient memory sharing where possible")

SYSTEM INVARIANTS

✓ 1. Immutability
   - Objects never change after creation
   - All 'modifications' create new objects
   - Parent pointers link old to new

✓ 2. Idempotency
   - Same inputs → same outputs
   - No side effects in kernel operations
   - Replaying operations yields same result

✓ 3. Content Addressing
   - Names are hashes of content
   - No randomness, no UUIDs
   - Deterministic across time and space

✓ 4. Evolution Model
   - Three states: In-Process → Current → History
   - Shift to future: commit current, merge in-process
   - History is in Git, not in memory

✓ 5. Kernel Statelessness
   - Kernel has no storage
   - Pure functions only
   - All state in HRT (managed by OS)

✓ 6. PyTorch Backend
   - Tensors support GPU acceleration
   - Clone-on-modify ensures immutability
   - Efficient memory sharing where possible


---

## Summary

This refactored system provides:

| Component | Responsibility | Key Property |
|-----------|---------------|--------------|
| **Kernel** | Pure morphisms | Stateless, deterministic |
| **HRT** | Operation data structure | Immutable, PyTorch-based |
| **Evolution** | State management | Three-state cycle |
| **Git** | Persistent storage | Content-addressed history |

### Evolution Cycle

```text
    ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
    │ In-Process  │ ──> │   Current   │ ──> │   History   │
    │ (new data)  │     │ (active)    │     │ (Git)       │
    └─────────────┘     └─────────────┘     └─────────────┘
           ↑                                         │
           └──────────── Shift to Future <───────────┘

    ingest → merge → commit
```

Ready for OS integration with Git-based persistent storage.